In [10]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import reciprocalspaceship as rs
import scipy.optimize as opt
import gemmi as gm
from tqdm import tqdm
import os
import pandas as pd
from pathlib import Path


import matplotlib
matplotlib.rc('xtick', labelsize=15) 
matplotlib.rc('ytick', labelsize=15)

In [2]:
from xtal_analysis.xtal_analysis_functions import *
from xtal_analysis.params import *

#### (1) Load in calculated_fobs, dark_fobs, light_fobs 

In [3]:
calc  = load_mtz("{path}/input_maps/FC_dark.mtz".format(path=loop_path))
off   = load_mtz("{path}/input_maps/neg5ps-400nm_fobs_unique1.mtz".format(path=loop_path))
on    = load_mtz("{path}/input_maps/{name}_fobs_unique1.mtz".format(path=loop_path, name=name))

##### Optional resolution cut

In [4]:
on    = res_cutoff(on, h_res, l_res)
off   = res_cutoff(off, h_res, l_res)
calc = res_cutoff(calc, h_res, l_res).copy(deep=True)

data     = pd.merge(calc, off, how='inner', right_index=True, left_index=True, suffixes=('_calc', '_off'))
data_all = pd.merge(data, on, how='inner', right_index=True, left_index=True, suffixes=('_off', '_on')).dropna()

In [5]:
data_all.head(1)

,,,FC_D,SIG_FC_D,PHI_D,dHKL_calc,FreeR_flag_off,F_off,SIGF_off,dHKL_off,FreeR_flag_on,F_on,SIGF_on,dHKL
H,K,L,,,,,,,,,,,,
0,0,4,72.729,1.0,180.0,18.0075,0,9.283087,0.95805734,18.0075,0,8.011057,0.8635788,18.0075


In [6]:
### this is in case there were NaN in on or off data
#calc.drop(calc.index[np.where(np.isnan(on))[0][0]], axis=0, inplace=True)

f_on    = np.array(data_all.F_on)
f_off   = np.array(data_all.F_off)
f_calc  = np.array(data_all.FC_D)

In [7]:
#### FOR LINEAR SCALING #####
#on_m, on_s   = scale(f_calc, f_on)
#off_m, off_s = scale(f_calc, f_off)
#sig_on       = data_all['SIGF_on']  / on_m
#sig_off      = data_all['SIGF_off'] / off_m


#### FOR ISOTROPIC RES SCALING #####
qs = 1/(2*data_all['dHKL'])

results_on, c_on, b_on, on_s     = scale_iso(f_calc, f_on,  np.array(data_all['dHKL']))
results_off, c_off, b_off, off_s = scale_iso(f_calc, f_off, np.array(data_all['dHKL']))

sig_on       = (c_on  * np.exp(-b_on*(qs**2)))  * data_all['SIGF_on']
sig_off      = (c_off * np.exp(-b_off*(qs**2))) * data_all['SIGF_off']

#### (3)  Generate q-weighted maps for a range of BDC values. Save each map as an mtz

#### AND

#### (4) Compute local and global correlation difference and save as list

In [8]:
Nbg_range = [0.97, 0.95] #test a couple of values before running 100
#Nbg_range = np.linspace(0,1, 100)


diffs     = []
Nbgs      = []
CC_locs   = []
CC_globs  = []

for Nbg in Nbg_range :
    diff, Nbg, CC_l, CC_g = screen_qweighted(on_s, off_s, sig_on, sig_off, calc, Nbg, name, alpha, h_res, l_res, loop_path, chrom_center, chrom_radius, sampling) 
    diffs.append(diff)
    Nbgs.append(Nbg)
    CC_locs.append(CC_l)
    CC_globs.append(CC_g)

#### (5) Tidy up! Move all generated files to new folder

In [11]:
#os.mkdir("Nbg_loop_qweighted/{}".format(name))
Path("Nbg_loop_qweighted/{}".format(name)).mkdir(parents=True, exist_ok=True)
os.system("mv {name}* Nbg_loop_qweighted/{name}/".format(name=name))

0

#### (6) Plot and find BDC that maximizes this difference

In [ ]:
fig, ax = plt.subplots(2,1, figsize=(8,7), tight_layout=True)

ax[0].plot(Nbgs, CC_locs, 'y', label='Local', linewidth=2)
ax[0].plot(Nbgs, CC_globs, 'k', label='Global', linewidth=2)

ax[1].plot(Nbgs, diffs, 'y', linestyle= 'dashed', label='Global - Local', linewidth=2)
ax[1].vlines(Nbgs[np.argmax(diffs)], 0.22, 0, 'r', linestyle= 'dashed', linewidth=1.5, label='Max={}'.format(np.round(Nbgs[np.argmax(diffs)], decimals=3)))

ax[0].set_title('{}'.format(name), fontsize=17)
ax[0].set_xlabel('N$_{\mathrm{bg}}$', fontsize=17)
ax[1].set_xlabel('N$_{\mathrm{bg}}$', fontsize=17)
ax[0].legend(fontsize=17)
ax[1].legend(fontsize=17)
plt.show()